Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### LR with L2 regularation ###


Regulartion cooefficient as "C"

In [25]:
batch_size = 128
C=1e-3
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + 
        C * tf.nn.l2_loss(weights)
    )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [19]:
num_steps=3001    
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we†l could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.563738
Minibatch accuracy: 4.7%
Validation accuracy: 8.7%
Minibatch loss at step 500: 2.935174
Minibatch accuracy: 71.1%
Validation accuracy: 76.0%
Minibatch loss at step 1000: 2.035929
Minibatch accuracy: 75.8%
Validation accuracy: 78.6%
Minibatch loss at step 1500: 1.171193
Minibatch accuracy: 82.8%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 0.891071
Minibatch accuracy: 82.0%
Validation accuracy: 80.3%
Minibatch loss at step 2500: 0.740657
Minibatch accuracy: 84.4%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.700905
Minibatch accuracy: 82.8%
Validation accuracy: 82.2%
Test accuracy: 88.9%


---
### 2 layer NN(relu) with L2 regularation ###

Regulartion cooefficient as "C"

In [29]:
def nn(dataset, weights, biases):
    # Training computation.
    layer_1 = tf.matmul(dataset, weights[0]) + biases[0] #shape (batch_size,relu_count)
    output = tf.matmul(tf.nn.relu(layer_1), weights[1]) + biases[1] #shape (batch_size,num_labels)
    return output, tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1])

batch_size = 128
relu_count = 1024 #hidden nodes count
C = 1e-3

neuralGraph = tf.Graph()
with neuralGraph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables. Weight dimension is only relevant with (dimension of each sample) * classes count
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, relu_count]))
    biases_1 = tf.Variable(tf.zeros([relu_count]))

    weights_2 = tf.Variable(tf.truncated_normal([relu_count, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))


    # Training computation.
    output, l2loss=nn(tf_train_dataset, [weights_1, weights_2], [biases_1, biases_2])
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels) + C*l2loss)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(output)
    valid_output, _ = nn(tf_valid_dataset, [weights_1, weights_2], [biases_1, biases_2])
    valid_prediction = tf.nn.softmax(valid_output)
    test_output, _ = nn(tf_test_dataset, [weights_1, weights_2], [biases_1, biases_2])
    test_prediction = tf.nn.softmax(test_output)


In [22]:
num_steps = 3001

with tf.Session(graph=neuralGraph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we†l could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 687.804016
Minibatch accuracy: 4.7%
Validation accuracy: 38.9%
Minibatch loss at step 500: 198.291611
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 116.660599
Minibatch accuracy: 77.3%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 68.955536
Minibatch accuracy: 85.9%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 41.439983
Minibatch accuracy: 87.5%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.222542
Minibatch accuracy: 87.5%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 15.430268
Minibatch accuracy: 90.6%
Validation accuracy: 87.1%
Test accuracy: 93.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

### LR with L2 regularation limited to `batch_cnt` 


In [26]:
num_steps=3001
batch_cnt = 3

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we†l could use better randomization across epochs.
        offset = ((step%batch_cnt) * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.827547
Minibatch accuracy: 10.2%
Validation accuracy: 14.4%
Minibatch loss at step 500: 1.836732
Minibatch accuracy: 100.0%
Validation accuracy: 66.1%
Minibatch loss at step 1000: 1.141201
Minibatch accuracy: 99.2%
Validation accuracy: 68.6%
Minibatch loss at step 1500: 0.701143
Minibatch accuracy: 100.0%
Validation accuracy: 70.3%
Minibatch loss at step 2000: 0.449972
Minibatch accuracy: 100.0%
Validation accuracy: 71.6%
Minibatch loss at step 2500: 0.301057
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 3000: 0.199047
Minibatch accuracy: 100.0%
Validation accuracy: 73.6%
Test accuracy: 80.2%


---
### 2 layer NN with L2 regularation limited to `batch_cnt` 


In [30]:
num_steps = 3001
batch_cnt = 3

with tf.Session(graph=neuralGraph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we†l could use better randomization across epochs.
        offset = (step%batch_cnt * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 749.649658
Minibatch accuracy: 10.9%
Validation accuracy: 39.2%
Minibatch loss at step 500: 190.520950
Minibatch accuracy: 100.0%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 115.542534
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Minibatch loss at step 1500: 70.071396
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Minibatch loss at step 2000: 42.495144
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Minibatch loss at step 2500: 25.771664
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Minibatch loss at step 3000: 15.630134
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Test accuracy: 82.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

### 2 layer NN (relu+dropout) with L2 regularation 


In [35]:
def nn3(dataset, weights, biases, use_dropout=False, keep_prob=tf.placeholder(tf.float32)):
    # Training computation.
    layer_1 = tf.matmul(dataset, weights[0]) + biases[0] #shape (batch_size,relu_count)
    layer_2 = tf.nn.relu(layer_1)
    if use_dropout:
        layer_2 = tf.nn.dropout(layer_2,keep_prob=keep_prob)
    output = tf.matmul(layer_2, weights[1]) + biases[1] #shape (batch_size,num_labels)
    return output, tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1])

batch_size = 128
relu_count = 1024 #hidden nodes count
C = 1e-3

neuralGraph3 = tf.Graph()
with neuralGraph3.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables. Weight dimension is only relevant with (dimension of each sample) * classes count
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, relu_count]))
    biases_1 = tf.Variable(tf.zeros([relu_count]))

    weights_2 = tf.Variable(tf.truncated_normal([relu_count, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))

    keep_prob = tf.placeholder(tf.float32)
    
    # Training computation.
    output, l2loss=nn3(tf_train_dataset, [weights_1, weights_2], [biases_1, biases_2], use_dropout=True, keep_prob=keep_prob)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels) + C*l2loss)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(output)
    valid_output, _ = nn3(tf_valid_dataset, [weights_1, weights_2], [biases_1, biases_2], use_dropout=False)
    valid_prediction = tf.nn.softmax(valid_output)
    test_output, _ = nn3(tf_test_dataset, [weights_1, weights_2], [biases_1, biases_2], use_dropout=False)
    test_prediction = tf.nn.softmax(test_output)


In [38]:
num_steps = 3001

with tf.Session(graph=neuralGraph3) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we†l could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 764.245972
Minibatch accuracy: 12.5%
Validation accuracy: 34.6%
Minibatch loss at step 500: 207.255402
Minibatch accuracy: 70.3%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 120.304611
Minibatch accuracy: 71.9%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 69.006424
Minibatch accuracy: 80.5%
Validation accuracy: 82.6%
Minibatch loss at step 2000: 41.827309
Minibatch accuracy: 80.5%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 25.370975
Minibatch accuracy: 87.5%
Validation accuracy: 84.7%
Minibatch loss at step 3000: 15.540739
Minibatch accuracy: 82.8%
Validation accuracy: 85.8%
Test accuracy: 92.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### 2 layer NN (relu+dropout) with L2 regularation 

Tuning `C` and steps

In [40]:
num_steps = 5000
C = 1e-4 #1e-3


with tf.Session(graph=neuralGraph3) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we†l could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 812.188538
Minibatch accuracy: 7.0%
Validation accuracy: 29.1%
Minibatch loss at step 500: 214.161041
Minibatch accuracy: 75.8%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 122.947876
Minibatch accuracy: 66.4%
Validation accuracy: 79.9%
Minibatch loss at step 1500: 69.072418
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Minibatch loss at step 2000: 41.944469
Minibatch accuracy: 77.3%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 25.422356
Minibatch accuracy: 85.9%
Validation accuracy: 84.8%
Minibatch loss at step 3000: 15.604239
Minibatch accuracy: 82.8%
Validation accuracy: 85.6%
Minibatch loss at step 3500: 9.688798
Minibatch accuracy: 79.7%
Validation accuracy: 85.9%
Minibatch loss at step 4000: 6.132636
Minibatch accuracy: 83.6%
Validation accuracy: 86.5%
Minibatch loss at step 4500: 3.740529
Minibatch accuracy: 88.3%
Validation accuracy: 86.7%
Test accuracy: 92.8%


In [41]:
num_steps = 3000
C = 1e-2 


with tf.Session(graph=neuralGraph3) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we†l could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 779.952271
Minibatch accuracy: 10.9%
Validation accuracy: 38.5%
Minibatch loss at step 500: 203.843933
Minibatch accuracy: 71.1%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 121.522591
Minibatch accuracy: 71.1%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 68.886444
Minibatch accuracy: 78.1%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 41.638256
Minibatch accuracy: 83.6%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 25.481361
Minibatch accuracy: 82.0%
Validation accuracy: 84.8%
Test accuracy: 91.8%


---
### 2 layer NN (relu+dropout) with L2 regularation and decayed learning rate

- Adding decayed learning rate

In [43]:
def nn3(dataset, weights, biases, use_dropout=False, keep_prob=tf.placeholder(tf.float32)):
    # Training computation.
    layer_1 = tf.matmul(dataset, weights[0]) + biases[0] #shape (batch_size,relu_count)
    layer_2 = tf.nn.relu(layer_1)
    if use_dropout:
        layer_2 = tf.nn.dropout(layer_2,keep_prob=keep_prob)
    output = tf.matmul(layer_2, weights[1]) + biases[1] #shape (batch_size,num_labels)
    return output, tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1])

batch_size = 128
relu_count = 1024 #hidden nodes count
C = 1e-3

neuralGraph3DecayedLearn = tf.Graph()
with neuralGraph3DecayedLearn.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables. Weight dimension is only relevant with (dimension of each sample) * classes count
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, relu_count]))
    biases_1 = tf.Variable(tf.zeros([relu_count]))

    weights_2 = tf.Variable(tf.truncated_normal([relu_count, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))

    keep_prob = tf.placeholder(tf.float32)
    
    # Training computation.
    output, l2loss=nn3(tf_train_dataset, [weights_1, weights_2], [biases_1, biases_2], use_dropout=True, keep_prob=keep_prob)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels) + C*l2loss)

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learning_rate=0.5, global_step=global_step, decay_steps=1000, decay_rate=0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(output)
    valid_output, _ = nn3(tf_valid_dataset, [weights_1, weights_2], [biases_1, biases_2], use_dropout=False)
    valid_prediction = tf.nn.softmax(valid_output)
    test_output, _ = nn3(tf_test_dataset, [weights_1, weights_2], [biases_1, biases_2], use_dropout=False)
    test_prediction = tf.nn.softmax(test_output)


In [44]:
num_steps = 3001
C = 1e-3


with tf.Session(graph=neuralGraph3DecayedLearn) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we†l could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 754.037842
Minibatch accuracy: 11.7%
Validation accuracy: 38.5%
Minibatch loss at step 500: 211.423584
Minibatch accuracy: 77.3%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 124.913963
Minibatch accuracy: 71.9%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 72.594589
Minibatch accuracy: 80.5%
Validation accuracy: 82.1%
Minibatch loss at step 2000: 45.205772
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 28.760923
Minibatch accuracy: 85.9%
Validation accuracy: 84.4%
Minibatch loss at step 3000: 18.445175
Minibatch accuracy: 81.2%
Validation accuracy: 85.7%
Test accuracy: 92.0%


### Tuning

- Tuning `C` and steps

In [45]:
num_steps = 5001
C = 1e-4


with tf.Session(graph=neuralGraph3DecayedLearn) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we†l could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 888.211304
Minibatch accuracy: 7.0%
Validation accuracy: 26.6%
Minibatch loss at step 500: 215.452072
Minibatch accuracy: 78.9%
Validation accuracy: 80.9%
Minibatch loss at step 1000: 126.055862
Minibatch accuracy: 72.7%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 72.905701
Minibatch accuracy: 83.6%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 45.367134
Minibatch accuracy: 82.0%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 28.637768
Minibatch accuracy: 85.9%
Validation accuracy: 84.6%
Minibatch loss at step 3000: 18.462023
Minibatch accuracy: 82.8%
Validation accuracy: 85.7%
Minibatch loss at step 3500: 12.255661
Minibatch accuracy: 82.8%
Validation accuracy: 86.2%
Minibatch loss at step 4000: 8.125489
Minibatch accuracy: 82.0%
Validation accuracy: 86.4%
Minibatch loss at step 4500: 5.325563
Minibatch accuracy: 90.6%
Validation accuracy: 86.8%
Minibatch loss at step 5000: 3.780099
Minibatch accuracy: 87.5%
Validati

In [46]:
num_steps = 8001
C = 1e-4


with tf.Session(graph=neuralGraph3DecayedLearn) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we†l could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 765.879883
Minibatch accuracy: 7.8%
Validation accuracy: 32.9%
Minibatch loss at step 500: 216.754120
Minibatch accuracy: 69.5%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 124.755630
Minibatch accuracy: 71.1%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 72.382034
Minibatch accuracy: 78.9%
Validation accuracy: 80.9%
Minibatch loss at step 2000: 45.583763
Minibatch accuracy: 81.2%
Validation accuracy: 82.5%
Minibatch loss at step 2500: 28.617577
Minibatch accuracy: 82.8%
Validation accuracy: 84.4%
Minibatch loss at step 3000: 18.359379
Minibatch accuracy: 85.9%
Validation accuracy: 85.4%
Minibatch loss at step 3500: 12.141620
Minibatch accuracy: 82.8%
Validation accuracy: 86.0%
Minibatch loss at step 4000: 8.094463
Minibatch accuracy: 85.2%
Validation accuracy: 86.3%
Minibatch loss at step 4500: 5.376963
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 5000: 3.818447
Minibatch accuracy: 85.2%
Validati